# EmoRecSys - CF

[EmoRecSys Survey](https://emorecsys.pt/)

###### Imports

In [1]:
# ---- INSTALLATIONS ---- #
# !pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357224 sha256=7f43039adb784633d2b892e5d447d07aaed5e547b4b9833257e4c483672fd2df
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [1]:
# ---- IMPORTS ---- #
from surprise import Reader, Dataset, CoClustering, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD, NMF
from ipywidgets import HBox, VBox, Image as WidgetImage
from sklearn.model_selection import ParameterGrid
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random

## 1. Preprocessing

In [2]:
def check_items(trainset, testset):
  """
  Checks if all items present in testset are also present in the trainset.
  
  Args:
    trainset (list of tuples): Each tuple represents a user-item interaction in the trainset. 
      Each tuple is expected to have the form (user, item, rating).
    testset (list of tuples): Each tuple represents a user-item interaction in the testset. 
      Each tuple is expected to have the form (user, item, rating).
    
  Returns:
    bool: True if all items in the testset are present in the trainset, otherwise returns False
  
  """
  trainset_items = set(list(item for _, item, _ in trainset))
  testset_items = set(list(item for _, item, _ in testset))

  items_unknown = list(testset_items - trainset_items)

  return len(items_unknown) == 0

def split_emorecsys(dataset, train=0.8, test=0.2):
  """
  Splits the dataset into training and testing sets, ensuring that each user's ratings are split accordingly.
  The function guarantees that each testset contains at least one relevant rating (liked item) and ensures
  that all items in the testset also appear in the trainset.

  Args:
    dataset (pandas.DataFrame): Dataframe containing user ratings. The dataframe must have columns `id_survey` (user ID),
      `id_photo` (item ID), and `like_bool` (binary rating indicating whether the user likes the item or not).
    train (float): The proportion of the dataset to include in the trainset. Default if 0.8.
    test (float): The proportion of the dataset to include in the testset. Default if 0.2.

  Returns:
    surprise_train (surprise.Trainset): A trainset formatted for use with the Surprise library.
    testset (list of tuples): A list of tuples representing the testset, where each tuple has the form (user, item, rating).
  
  """
  trainset, testset = list(), list()

  items_ratings = dataset.groupby('id_photo').size().to_dict() # get number of rating for each id_photo
  items_one_rating = [id_photo for id_photo, size in items_ratings.items() if size == 1] # get the ones with only 1 rating
  # print(items_one_rating)


  for user in list(set(dataset['id_survey'])):
    user_ratings = dataset[dataset['id_survey'] == user]
    all_ratings = list((user, id_photo, like_bool) for id_photo, like_bool in zip(user_ratings['id_photo'], user_ratings['like_bool']))
    size_ratings = len(all_ratings)

    user_trainset = list(rating for rating in all_ratings if rating[1] in items_one_rating) # start trainset with items with only one rating

    relevant = list(x for x in all_ratings if x[2] == 1)
    if relevant: # making sure that we have at least one relevant rating in the testset
      new_rating = random.sample(relevant, 1)
      user_testset = new_rating if new_rating[0] not in user_trainset else list()
    else:
      raise ValueError('The user ' + str(user) + ' did not liked any photo')

    while len(user_testset) < size_ratings*test: # add random ratings until fullfil the size of testset
      new_rating = random.choice(all_ratings)
      if new_rating not in user_testset and new_rating not in user_trainset:
        user_testset.append(new_rating)

    # adding remaining ratings to trainset
    user_trainset.extend(new_rating for new_rating in all_ratings if new_rating not in user_testset and new_rating not in user_trainset)

    assert len(user_trainset) == size_ratings*train
    assert len(user_testset) == size_ratings*test

    # print(user_trainset)
    # print(user_testset)

    trainset.extend(user_trainset)
    testset.extend(user_testset)

  if check_items(trainset, testset):
    train_df = pd.DataFrame(trainset, columns=['id_survey', 'id_photo', 'like_bool']) # like_bool
  else:
    return split_emorecsys(dataset, train, test)

  reader = Reader(rating_scale=(0,1)) # like_bool
  surprise_train = Dataset.load_from_df(train_df, reader).build_full_trainset()

  return surprise_train, testset

## 2. Modelling and Evaluating

In [12]:
models_grid = {'KNN': {'k': [10, 20, 30],
                       'min_k': [1, 2, 3],
                       'sim_options': [{'name': 'pearson', 'user_based': False},
                                       {'name': 'pearson', 'user_based': True},
                                       {'name': 'msd', 'user_based': False},
                                       {'name': 'msd', 'user_based': True}],
                       'verbose': [False]},

               'SVD': {'n_factors': [10, 50, 100],
                       'n_epochs': [20, 40, 60],
                       'lr_all': [0.005, 0.01, 0.02],
                       'reg_all': [0.05, 0.1 , 0.2]},

               'NMF': {'n_factors': [10, 50, 100],
                       'n_epochs': [20, 40, 60],
                       'reg_pu': [0.001, 0.005, 0.01],
                       'reg_qi': [0.01, 0.05, 0.1]},

               'CoClustering': {'n_cltr_u': np.arange(10, 101, 10),  # number of user clusters
                                'n_cltr_i': np.arange(10, 201, 10),  # number of item clusters
                                'n_epochs': np.arange(10, 51, 10)}} # number of epochs

In [9]:
dataset_cf = pd.read_csv("../data/csvs/ratings.csv")

##### Evaluation Functions


- vamos prever os itens do `testset` + x = 20, sendo x são items que o user AINDA NÃO VIU
- os items relevantes serão os que estão presentes no testset E TEM SCORE 1 (gostaram -- porque estamos a usar o like_bool)
  - para cada user, as previsões são ordenadas DESC com base no rating estimado `est`
  - depois as previsões são filtradas para manter apenas aquelas com `est` >= threshold (0.5), e serão essas as recomendações (até k).

In [3]:
def get_to_predict(user, trainset, testset):
  """
  Generates a list of items to be predicted for a given user. It includes items from the testset that the user has interacted
  with and randomly adds items the user has not seen, ensuring the total number of items is 20.
  
  Args:
    user (int or str): The user ID for whom the predictions are to be made.
    trainset (surprise.Trainset): The trainset formatted for use with the Surprise library.
    testset (list of tuples): A list of tuples representing the testset, where each tuple has the form (user, item, rating).
    
  Returns:
    items_testset (list of tuples): A list of tuples representing the items to predict for the given user, where each tuple has
    the form (user, item).
  
  """

  # get the items from the testset for given user
  items_testset = list((uid, iid) for uid, iid, _ in testset if uid == user)
  items_seen_by_user = {trainset.to_raw_iid(iid) for (uid, iid, _) in trainset.all_ratings() if trainset.to_raw_uid(uid) == user}
  all_items = {trainset.to_raw_iid(iid) for (_, iid, _) in trainset.all_ratings()}

  # find items user has not seen, and randomly sample 20 unseen items
  items_unseen_by_user = list(all_items - items_seen_by_user)
  unseen_sample = random.sample(items_unseen_by_user, 20)

  existing_iids = [iid for _, iid in items_testset]
  for item in unseen_sample:
    # add item if its not already in the list for prediction, while there's less than 20 items
    if (item not in existing_iids) and (len(items_testset) < 20):
      items_testset.append((user, item))
      existing_iids.append(item)

  # print(f'User {user} - Items to predict: {items_testset}')
  return items_testset

def precision_recall_at_k(model, trainset, testset, k=10, threshold=0):
  """
  Calculates the precision, recall and F1 score ate a specified cutoff k for eacg user in the testset. It evaluates a recommendation
  model's performance by generating predictions for items and comparing them to the actual relevant items.

  Args:
    model (object): The recommendation model used to make predictions.
    trainset (surprise.Trainset): The trainset formatted for use with the Surprise library.
    testset (list of tuples): A list of tuples representing the testset, where each tuple has the form (user, item, rating).
    k (int): The number of top recommendations to consider for each user. Default is 10.
    threshold (float): The minimum estimated rating value for an item to be considered recommended. Default is 0.

  Returns:
    precisions (dict): A dictionary where keys are user IDs and values are the precision at k for each user.
    recalls (dict): A dictionary where keys are user IDs and values are the recall at k for each user.
    f1scores (dict): A dictionary where keys are user IDs and values are the F1 score at k for each user.

  """
  precisions, recalls, f1scores = dict(), dict(), dict()
  # get list of users in testset
  test_users = list(set(item[0] for item in testset))

  for user in test_users:
    # generate items to predict for given user
    to_predict = get_to_predict(user, trainset, testset)
    predictions = [model.predict(uid, iid) for (uid, iid) in to_predict]
    # print(predictions)
    # print(f'Prediction for user {user}: {predictions}')

    ratings = [(iid, est) for (_, iid, _, est, _) in predictions if est >= threshold] # only recommend when est >= threshold
    ratings.sort(key=lambda x: x[1], reverse=True) # descending sort

    # relevant items in the testset need to have a true_rating > 0
    items_relevant = list(item[1] for item in testset if item[0] == user and item[2] > 0) # like_bool

    # print("USER", user)
    # print("ITENS RELEVANTES:", items_relevant)
    # for i in range(k):
    #   print(f"K = {i+1} - ", ratings[:i+1])
    # print()

    rel = len(items_relevant) # total number of relevant items to the user
    rel_rec = sum(np.isin(iid, items_relevant) for iid, _ in ratings[:k]) # number of relevant items recommended to the user

    precisions[user] = rel_rec / k # number of relevant items recommended to the user / total number of recommended items to the user
    recalls[user] = rel_rec / rel if rel != 0 else 1 # number of relevant items recommended to the user / total number of relevant items to the user
    f1scores[user] = (2*precisions[user]*recalls[user]) / (precisions[user]+recalls[user]) if (precisions[user]+recalls[user]) != 0 else 0.0

  return precisions, recalls, f1scores

def evaluation(model, trainset, testset, k=10):
  """
  Evaluates the performance of a recommendation model by calculating precision, recall and F1 scores for different values of k
  (the number of top recommendations). It returns the average precision, recall and F1 scores for all users at each value of k.

  Args:
    model (object): The recommendation model used to make predictions.
    trainset (surprise.Trainset): The trainset formatted for use with the Surprise library.
    testset (list of tuples): A list of tuples representing the testset, where each tuple has the form (user, item, rating).
    k (int): The number of top recommendations to consider for each user. Default is 10.

  Returns:
    precisions (list of tuples): A list of average precision scores at each value of k.
    recalls (list of tuples): A list of average recall scores at each value of k.
    f1scores (list of tuples): A list of average F1 scores at each value of k.

  """
  precisions, recalls, f1scores = [], [], []

  # for k in range(k, k+1):
  for k in range(1, k+1):
    # precision_k, recall_k, f1score_k = precision_recall_at_k(predictions, testset, k, threshold=0)
    precision_k, recall_k, f1score_k = precision_recall_at_k(model, trainset, testset, k, threshold=0.5) # like_bool

    precision = np.mean(list(precision_k.values())) # average value of precision score of all users at top k
    recall = np.mean(list(recall_k.values())) # average value of recall score of all users at top k
    f1score = np.mean(list(f1score_k.values())) # f1score of all users at top k

    precisions.append(precision)
    recalls.append(recall)
    f1scores.append(f1score)

    # print(f'Precision: {precision}, Recall: {recall}, F1 Score: {f1score}')

  return precisions, recalls, f1scores

def avg_metrics(precisions, recalls, f1scores, k):
  """
  Calculates the average precision, recall, and F1 scores over multiple evaluations for each value of k. It returns the rounded
  average metrics for each k value.

  Args:
    precisions (list of list of floats): A list where each element is a list of precision scores for different values of k from 
      multiple evaluations.
    recalls (list of list of floats): A list where each element is a list of recall scores for different values of k from 
      multiple evaluations.
    f1scores (list of list of floats): A list where each element is a list of F1 scores for different values of k from 
      multiple evaluations.
    k (int): The number of top recommendations considered for each user. This determines the length of the precision, recall,
      and F1 score lists.

  Returns:
    precision_avg (list of floats): A list of average precision scores for each value of k, rounded for four decimal places.
    recall_avg (list of floats): A list of average recall scores for each value of k, rounded for four decimal places.
    f1score_avg (list of floats): A list of average F1 scores for each value of k, rounded for four decimal places.
  
  """
  precision_avg, recall_avg, f1score_avg = [], [], []

  # for i in range(1):
  for i in range(k):
    precision = list(prec[i] for prec in precisions)
    recall = list(rec[i] for rec in recalls)
    f1score = list(f1[i] for f1 in f1scores)

    precision_avg.append(np.round(np.mean(precision), 4))
    recall_avg.append(np.round(np.mean(recall), 4))
    f1score_avg.append(np.round(np.mean(f1score), 4))

  return precision_avg, recall_avg, f1score_avg

def gridsearch_recommendation(model, param_grid, dataset, k=10):
  """
  Performs a grid search to find the best hyperparameters for a given recommendation model based on average precision, recall, 
  and F1 scores across multiple evaluations. It selects the best model on the highest F1 scores across multiple evaluations.
  It selects the best model based on the highest F1 score at a specified cutoff k.

  Args:
    model (class): The recommendation model class to be tunned.
    param_grid (dict): A dictionary where the keys are hyperparameters and the values are lists of parameter settings to try.
    dataset (pandas.DataFrame): Dataframe containing user ratings. The dataframe must have columns `id_survey` (user ID),
      `id_photo` (item ID), and `like_bool` (binary rating indicating whether the user likes the item or not).
    k (int): The number of top recommendations to consider for eacg user. Default is 10.

  Returns:
    Prints the best hyperparameters and their corresponding F1 score.
    Prints the precision, recall and F1 scores for the best model at each value of k from 1 to the specified k.
  
  """
  best_score = -1
  best_params = None

  for params in list(ParameterGrid(param_grid)):
    precisions_final, recalls_final, f1scores_final = [], [], []
    for i in range(5):
      trainset, testset = split_emorecsys(dataset)

      algo = model(**params)
      algo.fit(trainset)

      precisions, recalls, f1scores = evaluation(algo, trainset, testset, k=k)

      precisions_final.append(precisions)
      recalls_final.append(recalls)
      f1scores_final.append(f1scores)

    precision_avg, recall_avg, f1score_avg = avg_metrics(precisions_final, recalls_final, f1scores_final, k)

    # print(f'Parameters: {params}')
    # print()
    # print(f'Precisions: {precision_avg}, Recalls: {recall_avg}, F1 Scores: {f1score_avg}')

    # using f1score (k = 10) to select the best model as it is a harmonious average of precision and recall @ k
    if f1score_avg[-1] > best_score:
      best_score = f1score_avg[-1]
      best_params = params

      # keeping the precision, recall and f1score values for the best model
      best_precisions, best_recalls, best_f1scores = precision_avg, recall_avg, f1score_avg

  print(f'Best parameters: {best_params}')
  print(f'Best F1 Score: {best_score}\n')

  for i in range(k):
    print(f"K = {i+1} - Precision: {best_precisions[i]}, Recall: {best_recalls[i]}, F1 Score: {best_f1scores[i]}")

### 2.1. SVD

In [ ]:
gridsearch_recommendation(SVD, models_grid['SVD'], dataset_cf, k=10)

Best parameters: {'lr_all': 0.01, 'n_epochs': 20, 'n_factors': 10, 'reg_all': 0.05}
Best F1 Score: 0.2344

K = 1 - Precision: 0.2466, Recall: 0.1055, F1 Score: 0.1442
K = 2 - Precision: 0.2123, Recall: 0.18, F1 Score: 0.1894
K = 3 - Precision: 0.1975, Recall: 0.2571, F1 Score: 0.2172
K = 4 - Precision: 0.1837, Recall: 0.3249, F1 Score: 0.2284
K = 5 - Precision: 0.1806, Recall: 0.4006, F1 Score: 0.2427
K = 6 - Precision: 0.1779, Recall: 0.4699, F1 Score: 0.2524
K = 7 - Precision: 0.1637, Recall: 0.5055, F1 Score: 0.2421
K = 8 - Precision: 0.1613, Recall: 0.565, F1 Score: 0.2463
K = 9 - Precision: 0.1506, Recall: 0.5902, F1 Score: 0.236
K = 10 - Precision: 0.1464, Recall: 0.6393, F1 Score: 0.2344


### 2.2. NMF

In [ ]:
gridsearch_recommendation(NMF, models_grid['NMF'], dataset_cf, k=10)

Best parameters: {'n_epochs': 20, 'n_factors': 100, 'reg_pu': 0.001, 'reg_qi': 0.01}
Best F1 Score: 0.3466

K = 1 - Precision: 0.9718, Recall: 0.4728, F1 Score: 0.6117
K = 2 - Precision: 0.8245, Recall: 0.7374, F1 Score: 0.7538
K = 3 - Precision: 0.7223, Recall: 0.9331, F1 Score: 0.7929
K = 4 - Precision: 0.5417, Recall: 0.9331, F1 Score: 0.6689
K = 5 - Precision: 0.4334, Recall: 0.9331, F1 Score: 0.5788
K = 6 - Precision: 0.3611, Recall: 0.9331, F1 Score: 0.5102
K = 7 - Precision: 0.3096, Recall: 0.9331, F1 Score: 0.4562
K = 8 - Precision: 0.271, Recall: 0.9344, F1 Score: 0.4129
K = 9 - Precision: 0.2409, Recall: 0.9344, F1 Score: 0.3769
K = 10 - Precision: 0.2168, Recall: 0.9344, F1 Score: 0.3466


### 2.3. Co-Clustering

In [ ]:
gridsearch_recommendation(CoClustering, models_grid['CoClustering'], dataset_cf, k=10)

Best parameters: {'n_cltr_i': 10, 'n_cltr_u': 100, 'n_epochs': 30}
Best F1 Score: 0.2544

K = 1 - Precision: 0.584, Recall: 0.2685, F1 Score: 0.354
K = 2 - Precision: 0.4933, Recall: 0.4348, F1 Score: 0.4469
K = 3 - Precision: 0.3853, Recall: 0.5033, F1 Score: 0.4236
K = 4 - Precision: 0.3233, Recall: 0.5667, F1 Score: 0.4006
K = 5 - Precision: 0.2724, Recall: 0.5971, F1 Score: 0.3648
K = 6 - Precision: 0.2421, Recall: 0.6387, F1 Score: 0.3433
K = 7 - Precision: 0.2121, Recall: 0.6528, F1 Score: 0.3136
K = 8 - Precision: 0.1899, Recall: 0.6683, F1 Score: 0.2901
K = 9 - Precision: 0.1725, Recall: 0.6853, F1 Score: 0.2707
K = 10 - Precision: 0.1587, Recall: 0.7014, F1 Score: 0.2544


### 2.4. KNNBasic ("Not enough Neighbours")

In [ ]:
gridsearch_recommendation(KNNBasic, models_grid['KNN'], dataset_cf, k=10)

Best parameters: {'k': 20, 'min_k': 3, 'sim_options': {'name': 'pearson', 'user_based': False}, 'verbose': False}
Best F1 Score: 0.3532

K = 1 - Precision: 0.9399, Recall: 0.454, F1 Score: 0.5883
K = 2 - Precision: 0.8172, Recall: 0.7307, F1 Score: 0.7463
K = 3 - Precision: 0.7366, Recall: 0.9489, F1 Score: 0.8074
K = 4 - Precision: 0.5528, Recall: 0.9493, F1 Score: 0.6817
K = 5 - Precision: 0.4422, Recall: 0.9493, F1 Score: 0.5899
K = 6 - Precision: 0.3685, Recall: 0.9493, F1 Score: 0.5201
K = 7 - Precision: 0.3159, Recall: 0.9493, F1 Score: 0.4651
K = 8 - Precision: 0.2764, Recall: 0.9493, F1 Score: 0.4207
K = 9 - Precision: 0.2457, Recall: 0.9493, F1 Score: 0.384
K = 10 - Precision: 0.2211, Recall: 0.9493, F1 Score: 0.3532


### 2.5. KNNWithMeans

In [ ]:
gridsearch_recommendation(KNNWithMeans, models_grid['KNN'], dataset_cf, k=10)

Best parameters: {'k': 30, 'min_k': 3, 'sim_options': {'name': 'pearson', 'user_based': True}, 'verbose': False}
Best F1 Score: 0.3138

K = 1 - Precision: 0.8368, Recall: 0.3892, F1 Score: 0.5133
K = 2 - Precision: 0.7399, Recall: 0.6448, F1 Score: 0.6695
K = 3 - Precision: 0.6569, Recall: 0.8258, F1 Score: 0.7144
K = 4 - Precision: 0.4926, Recall: 0.8258, F1 Score: 0.6036
K = 5 - Precision: 0.3941, Recall: 0.8258, F1 Score: 0.5229
K = 6 - Precision: 0.3284, Recall: 0.8258, F1 Score: 0.4613
K = 7 - Precision: 0.2815, Recall: 0.8258, F1 Score: 0.4127
K = 8 - Precision: 0.2463, Recall: 0.8258, F1 Score: 0.3735
K = 9 - Precision: 0.219, Recall: 0.8258, F1 Score: 0.3411
K = 10 - Precision: 0.1971, Recall: 0.8258, F1 Score: 0.3138


### 2.6. KNNWithZScore

In [ ]:
gridsearch_recommendation(KNNWithZScore, models_grid['KNN'], dataset_cf, k=10)

Best parameters: {'k': 10, 'min_k': 3, 'sim_options': {'name': 'pearson', 'user_based': True}, 'verbose': False}
Best F1 Score: 0.3051

K = 1 - Precision: 0.8307, Recall: 0.3869, F1 Score: 0.5104
K = 2 - Precision: 0.7166, Recall: 0.6207, F1 Score: 0.6464
K = 3 - Precision: 0.6389, Recall: 0.8027, F1 Score: 0.6944
K = 4 - Precision: 0.4791, Recall: 0.8027, F1 Score: 0.5868
K = 5 - Precision: 0.3833, Recall: 0.8027, F1 Score: 0.5083
K = 6 - Precision: 0.3194, Recall: 0.8027, F1 Score: 0.4485
K = 7 - Precision: 0.2738, Recall: 0.8027, F1 Score: 0.4013
K = 8 - Precision: 0.2396, Recall: 0.8027, F1 Score: 0.3631
K = 9 - Precision: 0.213, Recall: 0.8027, F1 Score: 0.3316
K = 10 - Precision: 0.1917, Recall: 0.8027, F1 Score: 0.3051


### 2.7. KNNBaseline

In [ ]:
gridsearch_recommendation(KNNBaseline, models_grid['KNN'], dataset_cf, k=10)

Best parameters: {'k': 10, 'min_k': 1, 'sim_options': {'name': 'pearson', 'user_based': True}, 'verbose': False}
Best F1 Score: 0.2475

K = 1 - Precision: 0.5779, Recall: 0.2734, F1 Score: 0.3573
K = 2 - Precision: 0.4577, Recall: 0.4164, F1 Score: 0.422
K = 3 - Precision: 0.3542, Recall: 0.4732, F1 Score: 0.3938
K = 4 - Precision: 0.2893, Recall: 0.5174, F1 Score: 0.3616
K = 5 - Precision: 0.2464, Recall: 0.5511, F1 Score: 0.3325
K = 6 - Precision: 0.2207, Recall: 0.5871, F1 Score: 0.314
K = 7 - Precision: 0.1984, Recall: 0.6174, F1 Score: 0.2944
K = 8 - Precision: 0.1787, Recall: 0.6346, F1 Score: 0.2738
K = 9 - Precision: 0.1651, Recall: 0.6558, F1 Score: 0.2595
K = 10 - Precision: 0.1541, Recall: 0.6816, F1 Score: 0.2475


## 3. Recommending

In [4]:
dataset_photos = pd.read_csv("../data/csvs/photos.csv")
dataset_photos.set_index('id', inplace=True)

In [82]:
def display_image(show_list, n_show=3):
  """
  Displays a list of images in a grid format. It reads images from the specified file paths and displays them using widgets.

  Args:
    show_list (list of str): Alist of item IDs representing the images to be displayed.
    n_show (int): The number of images to display per row. Default is 3.

  """
  relevant_images_widgets = []
  for item in show_list:
    image = dataset_photos.loc[item]['file_name']
    ext = dataset_photos.loc[item]['ext']
    image_path = f'../data/photos/{image}.{ext}'

    # print(image)

    with open(image_path, "rb") as file:
      img = file.read()

    img_widget = WidgetImage(value=img, format='jpg', width=200, height=200)
    relevant_images_widgets.append(img_widget)

  display(VBox([HBox(relevant_images_widgets[i:i+n_show]) for i in range(0, len(relevant_images_widgets), n_show)]))

In [83]:
def recommending_cf(model, params, dataset):
  """
  Recommends images to a random user using a collaborative filtering model. It first display the relevant images for the user,
  then trains the model, makes predictions, and displays the top 5 recommended images.

  Args:
    model (class): The recommendation model class to used.
    params (dict): A dictionary of hyperparameters for the model.
    dataset (pandas.DataFrame): Dataframe containing user ratings. The dataframe must have columns `id_survey` (user ID),
      `id_photo` (item ID), and `like_bool` (binary rating indicating whether the user likes the item or not).
  
  """
  trainset, testset = split_emorecsys(dataset_cf)

  test_users = list(set(item[0] for item in testset))
  user = random.choice(test_users)
  items_relevant = list(item[1] for item in testset if item[0] == user and item[2] > 0)

  print('RELEVANT IMAGES:')
  display_image(items_relevant, len(items_relevant))

  algo = model(**params)
  algo.fit(trainset)

  to_predict = get_to_predict(user, trainset, testset)
  predictions = [algo.predict(uid, iid) for (uid, iid) in to_predict]

  ratings = [(iid, est) for (_, iid, _, est, _) in predictions if est >= 0.5]
  ratings.sort(key=lambda x: x[1], reverse=True)

  print('TOP 5 IMAGES RECOMMENDED:')
  ratings_5 = list(iid for iid, _ in ratings[:5])
  display_image(ratings_5, 5)

In [84]:
recommending_cf(model=NMF, params={'n_epochs': 20, 'n_factors': 100, 'reg_pu': 0.001, 'reg_qi': 0.01}, dataset=dataset_cf)

RELEVANT IMAGES:


TOP 5 IMAGES RECOMMENDED:
